In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "pass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'dog.jpg' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    
     html.Div( #div to display the image on the same line as the dashboard text
        style={
            'display': 'flex', #create flexbox
            'alignItems': 'center',  #align everything to the center
            'justifyContent': 'center',  
            'flexWrap': 'wrap'  #allows content to wrap on smaller screens
        }, 
        children=[
            html.Img( #display image to the left of header
                src=f'data:image/png;base64,{encoded_image.decode()}',
                style={
                    'height': '200px',  # change image height
                }
            ),
            html.Div(
                html.B(html.H1('CS-340 Dashboard')),  
                style={'display': 'flex', 'alignItems': 'center'} #enable flexbox layout and align center
            )
        ]
    ),
    html.Hr(),
    html.Div([
        dcc.Dropdown( #code for the dropdown menu
            id='filter-type', #id to return the value that is selected to the update database function
            options=[
                {'label':'Water Rescue', 'value': 'Water Rescue'},
                {'label':'Mountain or Wilderness Rescue','value':'Mountain or Wilderness Rescue'},
                {'label':'Disaster or Individual Tracking','value':'Disaster or Individual Tracking'}, 
                {'label':'No Filter','value':'No Filter'}
            ],
            value='No Filter' #makes the default value No Filter
        ),
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 
                         editable=False,         
                         filter_action="native", 
                         sort_action="native",   
                         sort_mode="multi",
                         column_selectable=False, 
                         row_selectable="single",     
                         row_deletable=False, 
                         selected_columns=[],
                         selected_rows=[0],   #selected row = 0 to avoid issues with no row selected         
                         page_action="native",
                         page_current=0,
                         page_size=10, 
                        ),
    html.Br(),
    html.Hr(),
   
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
     html.Div(
        html.Center(html.H1('Aaron Maciag Final Project'))
    )
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id','data'),Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    #Change query value based off of the dropdown search 
    #query for Water Rescue
    if filter_type == 'Water Rescue':
        query = {
    'breed': {'$in': ['Labrador Retriever', 'Chesapeake Bay Retriever', 'Newfoundland']},
    'sex_upon_outcome': 'Intact Female',
    'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        
    #query for mountain or wilderness rescue
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
    'breed': {'$in': ['German Sheperd', 'Alaskan Malamute', 'Old English Sheepdog','Siberian Husky','Rottweiler']},
    'sex_upon_outcome': 'Intact Male',
    'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        
    #query for Disaster or individual tracking
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
    'breed': {'$in': ['Doberman Pinscher','German Sheperd', 'Golden Retriever', 'Bloodhound','Rottweiler']},
    'sex_upon_outcome': 'Intact Male',
    'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
        
    #default returns entire database    
    else:
        query = {}
        
    df = pd.DataFrame.from_records(db.read(query)) #return data using query
    df.drop(columns=['_id'],inplace=True)#get rid of _id column
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    
    return data, columns
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


    # This callback will update the geo-location chart for the selected data entry
    # derived_virtual_data will be the set of data available from the datatable in the form of 
    # a dictionary.
    # derived_virtual_selected_rows will be the selected row(s) in the table in the form of
    # a list. For this application, we are only permitting single row selection so there is only
    # one value in the list.
    # The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        return [] #return default map if index is none
    else: 
        row = index[0]
    lat = float(dff.iloc[row]['location_lat']) #i was having issues with the code not selecting the long and lat
    long = float(dff.iloc[row]['location_long'])#this code stores the lattitude and lognitude in variables
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[33.4484,-112.0740], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # Column 13 and 14 define the grid-coordinates for 
                   # the map
                   # Column 4 defines the breed for the animal
                   # Column 9 defines the name of the animal
                   #dl.Marker(position=[float(dff.iloc[row,13]),float(dff.iloc[row,14])],#needed to change to 14 and 15 and convert to float
                   dl.Marker(position=[lat,long],#the provided code didnt work so i had to use variables and search by column name in order to make the code work
                             children=[
                                 dl.Tooltip(dff.iloc[row]['name']),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row]['name'])
                                 ])
                             ])
               ])
    ]



app.run_server(debug=True)


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
